In [1]:
import pandas as pd
import bioframe
import os
import dask.dataframe as dd
import numpy as np

mm10-cCREs_ESC_DNase_merged5kb_mm10.bed generated in mm10_CREs_ESC.ipynb

In [2]:
cres = pd.read_csv('mm10-cCREs_ESC_DNase_merged5kb_mm10.bed', 
                   names=['chrom', 'start', 'end'], sep="\t", 
                   dtype={'chrom': str, 'start': int, "end": int},
                   usecols = [i for i in range(3)])

remap2022_all_macs2_mm10_v1_0.bed downloaded from https://remap.univ-amu.fr/storage/remap2022/mm10/MACS2/remap2022_all_macs2_mm10_v1_0.bed.gz

In [3]:
ESC_names = ["G4", "ZHBTc4", "CCE", "mesc", "ES-Bruce4", "EpiSC", "ES-E14", "mESC-E14", 
             "EpiLC", "E14TG2a-4", "E14", "mESC"]

In [4]:
#Load ReMap2022 data in chunks
chunks = pd.read_csv('remap2022_all_macs2_mm10_v1_0.bed', 
                    sep="\s+|\.", header=None, engine="python", usecols = [i for i in range(6)],
                    names=["chrom", "start", "end", "ID", "Factor", "Cell_type"],
                    dtype={'chrom': str, 'start': int, "end": int,
                           "ID": str, "Factor": str, "Cell_type": str}, 
                    chunksize=10**6)
remap = pd.DataFrame()
for chunk in chunks:
    chunk = chunk[chunk["Cell_type"].isin(ESC_names)]
    remap = pd.concat([remap, chunk])

In [5]:
#Overlap with CREs
remap = bioframe.overlap(cres, remap, how='left', suffixes=['_cres', '']).fillna(0)

In [6]:
remap = remap.rename(columns = {"ID": "GSEID"})

In [7]:
remap["ID_Factor"] = remap["GSEID"].astype(str) + "_" + remap["Factor"].astype(str)

In [8]:
remap = remap[remap["ID_Factor"] != "0_0"]
remap = remap.dropna()

In [9]:
#Renaming some datasets that are the same but have different names in ReMap2022 and cistromeDB
remap["ID_Factor"] = np.where(remap["ID_Factor"] == "ENCSR000ERX_HCFC1", "GSE36030_HCFC1", remap["ID_Factor"])
remap["ID_Factor"] = np.where(remap["ID_Factor"] == "ENCSR604XDL_MAFK", "GSE36030_MAFK", remap["ID_Factor"])
remap["ID_Factor"] = np.where(remap["ID_Factor"] == "ENCSR000ERW_ZC3H11A", "GSE36030_ZC3H11A", remap["ID_Factor"])

Manually annotated blacklist to remove datasets with perturbations in cistromeDB

In [10]:
blacklist = [8980,       #MEN1 -> H3K4me3 @ MEN1 KO
             72156, 52674, #c17orf96 -> H3K4me3 @ c17orf96 KD
             68937, # ZFP57 -> KAP1 @ ZFP57 KO
             59471, 59472, 59473, 59474, # ZFP57 -> H3K27ac @ ZFP57 WT/KO
             49151, # Liver
             # CTCF
             51815, #D3 LIF withdrawal, not really ES cells
             93023, 93024, 93030, 93033, 93032, 94651, 94662, # Deleted from GEO
             85175, 85176, # AID, AUX treated
             81237, #MLL3/4 KO
             5935, 5936, #TKO
             104077, #DKO
             # RAD21
             86507, 87703, 87434, 81689, 83522, 85629, 86510, 83519, #DKO or MLL3/4 KO, or catalytically dead
             4501, #Embryoid bodies
             55974, 55966, 55962, #SA1 - brain, pancreas, adult cortex
             32902, 55976, 55968, 55964, 55978, # SMC1A - FLV treated, brain, pancreas, adult cortex, SA1 KO brain
             34713, 37756, 41352, 41353, 41354, 41356, 41357, 41833, 47343, 47344, 47767, 47768, 47769, 67363, 81951, 83510, 67540, 67545, #Knockdown conditions
             37751, 44006, 44007, 67571, 67572, #Histones
             43845, 48363, 48364, 50202, 58925, 58926, 68336, 68340, 71071, 71075, #KO conditions
             73463, 73465, 73469, 74264, 74266, 75371, 75372, 76746, 81310, 72437, 72441, 72445, 73263, 73264, 73285, #KO conditions
             8539, 48513, 48514, 48515, 48516, 71177, 71181, 71183, #KO conditions
             2866, 2867, 2868, 2869, 2879, 2880, 37689, 51388, 51389, 53553, 59476, 72455, 73003, 73005,  #shRNA
             85174, #AID
             33721, 33755, 33762, 33765, 33779, 33790, 33794, 36800, 36816, 36826, 36828, 36829, 36831, 71698, 72457, 54539, 56193, 71072, 71892, 71895, 71897, 71898, 73286, 73287, 73288, 75369, 75370, 86476, 83868, 83869, #Mutants
             71172, 71174, 71178, 71179, 71180, 71891, 75197, 75219, 75220,  #flox
             53113, 87245, 32907, 34083, 67353, 67354, 48940, 48938, 48943, 48945, 48948, 48949, 48950, 53944, 53947, 53950, 53951, 68444, 74265, #Treatments
             4540, 59428, 2870, 2871, 55147, 34528, 34540, 34548, 44552, 44553,  75579, 75787, #Transgene
             55144, 55145, 55146, 43996, 82629, 82626, 82624, #Differentiated
             44554, 44556, 44557, 44533, 44534, 44535, 44536, 44560, #sgRNA
             51971, 54581, 54582, 54583, 55872, #not ES
             76350, 76353, 76347, #Mitotic
             ]

mouse_factor files downloaded as batch from cistromeDB. mouse_factor_ES.txt generated using GEO_query_ES.R

In [11]:
factors = pd.read_csv('mouse_factor_ES.txt', sep='\t')
#Quality filtering according to cistromeDB guidelines
factors = factors[(factors['FastQC'] > 25) & (factors['UniquelyMappedRatio'] > 0.6) & (factors['PBC'] > 0.8) &
                  (factors['PeaksFoldChangeAbove10'] > 500) & (factors['FRiP'] > 0.01) & (factors['PeaksUnionDHSRatio'] > 0.7)]
#Remove blacklist files
es = factors[(factors['Cell_type']=='Embryonic Stem Cell') & (~factors['DCid'].isin(blacklist))]

In [12]:
namedict_es = dict(zip(es['DCid'], es['Factor']))
es_factortable = pd.concat([bioframe.read_table(f'mouse_factor/{fid}_sort_peaks.narrowPeak.bed', 
                                                schema='narrowPeak').assign(id=fid, factor=factor).rename(
    columns={'chrom':'peak_chrom', 'start':'peak_start', 'end':'peak_end'}) for fid, factor in namedict_es.items()])

In [13]:
es_factortable = bioframe.overlap(cres, es_factortable, how='left', suffixes=['_cres', ''], cols2=['peak_chrom', 'peak_start', 'peak_end']).fillna(0)
es_factortable[['start_cres', 'end_cres', 'peak_start','peak_end']] = es_factortable[['start_cres', 'end_cres', 'peak_start','peak_end']].astype(int)

In [14]:
es_factortable = es_factortable.dropna()

In [15]:
es_factortable = es_factortable[["chrom_cres", "start_cres", "end_cres", "factor", "id", 
                                 "peak_chrom", "peak_start", "peak_end"]]

In [16]:
es_factortable.columns = ["chrom_cres", "start_cres", "end_cres", "Factor", "DCid",
                          "chrom", "start" , "end"]

In [17]:
es_factortable["DCid"] = es_factortable["DCid"].astype(int)

In [18]:
es_factortable = pd.merge(es_factortable, factors[["DCid", "GSEID"]], on="DCid")

In [19]:
es_factortable["ID_Factor"] = es_factortable["GSEID"].astype(str) + "_" + es_factortable["Factor"].astype(str)

In [20]:
#Combine cistromeDB and ReMap2022
cistrome_remap = pd.concat([remap[["chrom_cres", "start_cres", "end_cres", "ID_Factor", "Factor"]], 
                            es_factortable[["chrom_cres", "start_cres", "end_cres", "ID_Factor", "Factor"]]])

In [21]:
cistrome_remap["coords_cre"] = cistrome_remap["chrom_cres"] + "_" + cistrome_remap["start_cres"].astype(str) + "_" + cistrome_remap["end_cres"].astype(str)

In [22]:
cistrome_remap = cistrome_remap.drop_duplicates(subset=["coords_cre", "ID_Factor"])

In [23]:
cistrome_remap.to_csv("mm10-cCREs_ESC_DNase_merged5kb_mm10_cistromeDB_ReMap2020_filtered.bed", sep="\t", index=False, header=True)